In [58]:
import numpy as np
import pandas as pd

In [59]:
header = ['userID','itemID','rating','timestamp']
df = pd.read_csv('train.csv')
print(df)

       userID  itemID  rating   timestamp
0        2409     256     5.0  1114232409
1        2533    1116     4.0  1351541044
2         422     369     4.0  1068357977
3         840      53     4.0  1010512429
4         521     167     3.5  1107571443
...       ...     ...     ...         ...
97837    2606     703     4.5  1125191720
97838    1338     490     2.0   939134372
97839    2081    1115     2.0  1094481868
97840    1942     597     3.0  1122004393
97841    2219      55     4.5  1341922541

[97842 rows x 4 columns]


In [60]:
n_users = df.userID.unique().shape[0]
n_items = df.loc[:,['itemID']].max()['itemID']
#print(n_items)
#print(type(n_items))
#print('number of user is '+ str(n_users) + ' | Number of item = ' + str(n_items))

4124


In [24]:
#from sklearn import model_selection as ms
#train_data,test_data = ms.train_test_split(df, test_size = 0.25)
#print(df)
#print(train_data)
#print(type(train_data))
#print(train_data.shape)
#print(test_data.shape)

In [61]:
train_data_matrix = np.zeros((n_users,n_items))
for line in df.itertuples():
    #print(line)
    train_data_matrix[int(line[1])-1, int(line[2])-1] = line[3]
#print(train_data_matrix)
train_data_matrix[:,0]+=1e-9
train_data_matrix[0]+=1e-9
#test_data_matrix = np.zeros((n_users, n_items))
#for line in test_data.itertuples():
#    test_data_matrix[line[1]-1, line[2]-1] = line[3]
#test_data_matrix[:,0]+=1e-9
#test_data_matrix[0]+=1e-9

In [9]:
#from sklearn.metrics.pairwise import pairwise_distances
#user_similarity = pairwise_distances(train_data_matrix, metric = "cosine")
#item_similarity = pairwise_distances(train_data_matrix.T, metric = "cosine")
#print(user_similarity)

In [30]:
import time
from tqdm import tqdm

In [35]:
def count_user_similarity(train_data_matrix):
    num_user=train_data_matrix.shape[0]
    pss_user=np.ones((num_user,num_user))
    mean_user = np.average(train_data_matrix,1,weights=np.int64(train_data_matrix>0))#求平均值
    for i in tqdm(range(num_user)):#进度条
        for j in range(i,num_user):
            if i==j :
                pss_user[i][j]=1
            else :
                #筛选共同打分的item索引
                ind = np.nonzero(train_data_matrix[i]*train_data_matrix[j])
                if len(ind[0])==1:
                    pss_user[i][j]=0
                    continue
                #计算相似度
                pss_user[i][j] = pss_user[j][i] = np.sum((train_data_matrix[i][ind]-mean_user[i])*(train_data_matrix[j][ind]-mean_user[j]))\
                                    /(np.sqrt(np.sum(np.square(train_data_matrix[i][ind]-mean_user[i])))*np.sqrt(np.sum(np.square(train_data_matrix[j][ind]-mean_user[j]))))
    return pss_user;

In [36]:
pss_user = count_user_similarity(train_data_matrix)
print(pss_user)

100%|██████████| 2967/2967 [03:13<00:00, 15.35it/s] 

[[ 1.00000000e+00 -2.58058655e-02 -1.52544074e-08 ... -2.04576184e-08
   7.01865093e-02  2.97128329e-01]
 [-2.58058655e-02  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   7.02270566e-01 -1.85436195e-01]
 [-1.52544074e-08  1.00000000e+00  1.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [-2.04576184e-08  1.00000000e+00  1.00000000e+00 ...  1.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.01865093e-02  7.02270566e-01  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  2.22824800e-01]
 [ 2.97128329e-01 -1.85436195e-01  1.00000000e+00 ...  1.00000000e+00
   2.22824800e-01  1.00000000e+00]]


In [37]:
def count_item_similarity(train_data_matrix):
    train_data_matrix_T = np.transpose(train_data_matrix)
    num_item=train_data_matrix_T.shape[0]
    print(num_item)
    pss_item=np.ones((num_item,num_item))
    #mean_item = np.average(train_data_matrix,0,weights=np.int64(train_data_matrix>0))#求平均值
    #print(mean_item)
    for i in tqdm(range(num_item)):
        for j in range(i,num_item):
            if i==j :
                pss_item[i][j]=1
            else :
                #筛选共同打分的item索引
                ind = np.nonzero(train_data_matrix_T[i]*train_data_matrix_T[j])
                #if len(ind[0])==1:
                #    pss_item[i][j]=0
                    continue
                #计算相似度
                pss_item[i][j] = pss_item[j][i] = np.sum((train_data_matrix_T[i][ind])*(train_data_matrix_T[j][ind]))\
                                    /(np.sqrt(np.sum(np.square(train_data_matrix_T[i][ind])))*np.sqrt(np.sum(np.square(train_data_matrix_T[j][ind]))))
    return pss_item;

In [38]:
pss_item = count_item_similarity(train_data_matrix)
print(pss_item)

  0%|          | 0/4124 [00:00<?, ?it/s]

4124


100%|██████████| 4124/4124 [07:30<00:00,  9.15it/s] 

[[1.         0.44570899 0.27418737 ... 0.4472136  0.         0.36771366]
 [0.44570899 1.         0.94657329 ... 0.83205029 0.         0.9587289 ]
 [0.27418737 0.94657329 1.         ... 0.         0.         0.94906888]
 ...
 [0.4472136  0.83205029 1.         ... 1.         0.         1.        ]
 [1.         1.         1.         ... 1.         1.         0.        ]
 [0.36771366 0.9587289  0.94906888 ... 1.         1.         1.        ]]


In [39]:
print(pss_item.shape)

(4124, 4124)


In [62]:
def mean_nozero(ratings, kind):
    if kind == 'user':
        user_bias = np.zeros(ratings.shape[0])
        for i in range(0, ratings.shape[0]):
            number = amount = 0
            for j in range(0, ratings.shape[1]):
                if(ratings[i,j] != 0):
                    number += 1
                    amount += ratings[i,j]
            user_bias[i] = amount/number
        return user_bias
    if kind == 'item':
        item_bias = np.zeros(ratings.shape[1])
        for i in range(0, ratings.shape[1]):
            number = amount = 0
            for j in range(0, ratings.shape[0]):
                if(ratings[j,i] != 0):
                    number += 1
                    amount += ratings[j,i]
            item_bias[i] = amount/number
        return item_bias

In [117]:
def predict_topk_nobias(ratings, similarity, kind='user', k=40):
    tmp1 = np.zeros(ratings.shape)
    tmp2 = np.zeros(ratings.shape[0])
    pred = np.zeros(ratings.shape)
    if kind == 'user':
        #user_bias = ratings.mean(axis=1)#按行求平均值
        user_bias = mean_nozero(ratings, 'user')
        ratings = (ratings - user_bias[:, np.newaxis]).copy()#user的打分减去其打分集合的平均分
#         for i in range(ratings.shape[0]):
#             for j in range(ratings.shape[1]):
#                 for m in range(ratings.shape[0]):
#                     if(ratings[m][j] < 1e-5):
#                         continue
#                     tmp1[i][j] += similarity[i,m]*ratings[m,j]
#                     tmp2[i] += similarity[i,m]
        tmp1 = similarity@np.vectorize(lambda x: 0 if x < 1e-6 else x)(ratings)
        tmp2 = np.sum(similarity@np.vectorize(lambda x : 0 if x < 1e-6 else 1 (ratings), axis = 1))
        for k in range(ratings.shape[0]):
            pred[k] = tmp1[k]/tmp2[k]
        pred+=user_bias[:,np.newaxis]
    if kind == 'item':
        #item_bias = ratings.mean(axis=0)
        item_bias = mean_nozero(ratings, 'item')
        print(item_bias)
        ratings = (ratings - item_bias[np.newaxis, :]).copy()
        for j in range(ratings.shape[1]):
            top_k_items = [np.argsort(similarity[:,j])[:-k-1:-1]]
            for i in range(ratings.shape[0]):
                for m in range(len(top_k_users)):
                    if(ratings[i][j] < 1e-5 or all(ratings[top_k_users[m]][j]) < 1e-5):
                        del top_k_users[m]
                pred[i, j] = similarity[j, :][top_k_items].dot(ratings[i, :][top_k_items].T) 
                pred[i, j] /= np.sum(np.abs(similarity[j, :][top_k_items])) 
        pred += item_bias[np.newaxis, :]
        
    return pred

In [43]:
def predict(rating, similarity, type = 'user'):
    if(type == 'user'):
        mean_user_rating = rating.mean(axis = 1)
        print(mean_user_rating)
        rating_diff = (rating - mean_user_rating[:,np.newaxis])
        #pred = mean_user_rating[:,np.newaxis] + similarity.dot(rating_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        print(rating.shape)
        print(similarity.shape)
        #pred = rating.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    #return pred

In [118]:
user_based_pred = predict_topk_nobias(train_data_matrix,pss_user, 'user',40)
print(user_based_pred)

TypeError: __init__() got an unexpected keyword argument 'axis'

In [269]:
print(user_based_pred.shape)

(2967, 4124)


In [270]:
df_user_result = pd.DataFrame(user_based_pred)
#print(df_result)
df_user_result.to_csv('user_based_result.csv',index = None)

In [274]:
item_based_pred = predict_topk_nobias(train_data_matrix,pss_item,'item',40)
print(item_based_pred)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


[[ 0.19224147  0.42063004  0.22049574 ... -0.1555696  -0.01494646
   0.38140799]
 [ 0.19234147  0.42073004  0.22059574 ... -0.1554696  -0.01484646
   0.38150799]
 [ 0.19224147  0.42063004  0.22049574 ... -0.1555696  -0.01494646
   0.38140799]
 ...
 [ 0.19224147  0.42063004  0.22049574 ... -0.1555696  -0.01494646
   0.38140799]
 [ 0.19224147  0.42063004  0.22049574 ... -0.1555696  -0.01494646
   0.38140799]
 [ 0.25869968  0.55495623  0.37946964 ... -0.0555696  -0.01494646
   0.48140799]]


In [ ]:
f = open('predict2.csv','w')
f.write('dataID,rating\n')
for i in range(test_data_matrix.shape[0]):
    predict_matrix[i][0] = i
    predict_matrix[i][1] = res[test_data_matrix[i][0]][test_data_matrix[i][1]]
    f.write('%d,%.8f\n'%(predict_matrix[i][0],predict_matrix[i][1]))

In [277]:
df_item_result = pd.DataFrame(item_based_pred)
#print(df_result)
df_item_result.to_csv('item_based_result.csv',index = None)

In [155]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [258]:
df_test = pd.read_csv('test_index.csv')
print(df_test)#这里为什么没有打分？

       userID  itemID
0        2345     468
1         381     181
2        2679     320
3        1638     632
4        1562      40
...       ...     ...
39674     886     273
39675     332      10
39676    2966     133
39677    2855     158
39678    2748      40

[39679 rows x 2 columns]


In [275]:
#n_users_test = df.userID.unique().shape[0]
#n_items_test = df.loc[:,['itemID']].max()['itemID']
#test_data_matrix = np.zeros(n_users, n_items)
#for line in df_test.itertuples():
#    test_data_matrix[int(line[1])-1, int(line[2])-1] = 
#precise = rmse(user_based_pred,test_data_matrix)
#print(precise)

In [276]:
#df_test = df_test.values
#precise = rmse(user_based_pred,test_data_matrix)
#print(precise)